# Masterthesis
## Data Loading and Preparation

#### Julian Jetz

### Inhalt:

* [Laden der CSV-Dateien](#csv)
* [Löschen der sonstigen Unfalltypen](#sonstige)
* [Unfallartendeklaration](#unfallart)
* [Unfallurschaendeklaration](#ursache)
* [Berechnung der beteiligten Lokomotiven](#Lok)
* [Berechnung der Zuggeschwindigkeit](#spd)
* [Berechnung des Infrastrukturschadens](#dmg)
* [Deklaration der Wettertypen](#weather)
* [Deklaration der Sichtbarkeit](#vis)
* [Deklaration der Streckenart](#Streckenart)
* [Deklaration der Staaten](#state)
* [Umrechnung des Schadens in Euro](#euro)
* [Bestimmung der Signifikanz des Unfalls](#sig)
* [Clusterung der Temperatur](#temp)
* [Reduzierung der MissingValues in der Spalte Streckenklasse](#trkcls)
* [Überprüfung, ob es zu einer Geschwindigkeitsüberschreitung kam](#spd_too_high)
* [Bestimmung der Jahreszeit](#jahreszeit)
* [Bestimmung der Zugart](#EQUIPMENT_TYPE)
* [Ersetzen der MissingValues in der Spalte Zuggewicht](#TONS)
* [Entfernung aller MissingValues Werte](#nan)
* [Aufteilen der Datensätze bei mehreren Zügen](#entfernen_mit_mehr_zügen)
* [Mehrere beteiligte Züge](#multi)

In [1]:
import glob
import numpy as np
import math
import pandas as pd
from currency_converter import CurrencyConverter
from datetime import date, datetime

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer as MICE

In [2]:
# N/A Werte werden deklariert
na_values = ['nan', 'N/A', 'NaN', 'NaT']
# Deklaration der Spalten, die geladen warden sollen 
cols = ['INCDTNO', 'YEAR4', 'MONTH', 'DAY', 'ACCDMG', 'EQPDMG', 'TRKDMG', 'TYPE', 'TYPEQ', 'TRKCLAS', 'TYPTRK', 'TRNSPD', 'HIGHSPD', 'ACCAUSE', 'HEADEND1', 'MIDMAN1', 'MIDREM1', 'RMAN1', 'RREM1', 'HEADEND2', 'MIDMAN2', 'MIDREM2', 'RMAN2', 'RREM2', 'LOADF1', 'LOADP1', 'EMPTYF1', 'EMPTYP1', 'LOADF2', 'EMPTYF2', 'LOADP2', 'EMPTYP2', 'TEMP', 'TONS', 'VISIBLTY', 'WEATHER', 'STATE', 'RREMPKLD', 'PASSKLD', 'OTHERKLD', 'RREMPINJ', 'PASSINJ', 'OTHERINJ']

### Laden der CSV <a class="anchor" id="csv"></a>

In [3]:
# Funktion zum Laden und Zusammenfügen der CSV Dateien
def get_merged_csv(flist, **kwargs):
    return pd.concat([pd.read_csv(f, **kwargs) for f in flist], ignore_index=True)

In [4]:
# Laden der CSV Dateien als ein Pandas DataFrame, Festlegen N/A Werte, Festlegen zu ladende Spalten, keine Index Spalte bei den Daten und Deklaration der Datentypen
df = get_merged_csv(glob.glob('csvfiles/*.csv'), na_values=na_values, index_col=False, usecols = cols, dtype = {"STATE" : "str", "TYPE" : "str", "TYPEQ" : "str", "WEATHER" : "str", "VISIBLTY" : "str", "TRKCLAS" : "str"})
df.shape[0]

116210

In [5]:
#df = get_merged_csv(glob.glob('csvfiles/*.csv'), na_values=na_values, index_col=False, usecols = cols, dtype = {"STATE" : "str", "TYPE" : "str", "TYPEQ" : "str", "WEATHER" : "str", "VISIBLTY" : "str", "YEAR4" : "int64", "MONTH" : "int64", "DAY" : "int64", "TRKCLAS" : "str"})
#df.shape[0]

#### Löschen der Unfalltypen Sonstige <a class="anchor" id="sonstige"></a>
In diesem Schritt werden die Unfalltypen mit dem Typ Sonstige aus dem Datensatz entfernt

In [6]:
df.drop(df.loc[df['TYPE']=="12"].index, inplace=True)
df.drop(df.loc[df['TYPE']=="13"].index, inplace=True)
df.drop(df.loc[df['TYPEQ']=="9"].index, inplace=True)

In [7]:
df.shape[0]

96491

#### Unfallartendeklaration <a class="anchor" id="unfallart"></a>

In [8]:
def getAccidentType(acctype):
    switcher = {
        "01" : "Entgleisung",
        "02" : "Kollision",
        "03" : "Kollision",
        "04" : "Kollision",
        "05" : "Kollision",
        "06" : "Kollision",
        "07" : "Bahnübergang",
        "08" : "Bahnübergang",
        "09" : "Kollision",
        "10" : "Fahrzeugbrand / Explosion",
        "11" : "Fahrzeugbrand / Explosion",
        
    }
    return switcher.get(acctype,"NaN")

In [9]:
df["ACCIDENT_TYPE"] = df.apply(lambda row : getAccidentType(row["TYPE"]), axis=1)

In [10]:
df["ACCIDENT_TYPE"].value_counts()

Entgleisung                  71021
Kollision                    16463
Bahnübergang                  7777
Fahrzeugbrand / Explosion     1230
Name: ACCIDENT_TYPE, dtype: int64

#### Laden der Daten mit den Unfallursachencluster und zuweisen dieser zu dem Dataset <a class="anchor" id="ursache"></a>

In [11]:
acc_causes = pd.read_csv('resources/ACC_CAUSE_LIST.csv',sep=';', na_values=na_values, encoding='latin1',index_col='ACCAUSE')

In [12]:
#Funktion zum Umwandeln der Unfallursache in die Unfall-ursache auf dem Abstraktionslevel 1
def getAccCauseLvl1(accause):
    return acc_causes.loc[accause]['ACCAUSE_LVL1']

In [13]:
#Funktion zum Umwandeln der Unfallursache in die Unfall-ursache auf dem Abstraktionslevel 1
def getAccCauseLvl2(accause):
    return acc_causes.loc[accause]['ACCAUSE_LVL2']

In [14]:
df=df.dropna(subset=['ACCAUSE'])

In [15]:
df['ACCCAUSE_LVL1']=df.apply(lambda row : getAccCauseLvl1(row['ACCAUSE']),axis=1)

In [16]:
df['ACCCAUSE_LVL2']=df.apply(lambda row : getAccCauseLvl2(row['ACCAUSE']),axis=1)

In [17]:
print('Unique Unfallursachen: %s & (neue) Unique Unfallursachencluster (Ursachen Level 1): %s & (neue) Unique Unfallursachencluster (Ursachen Level 2): %s' % (df['ACCAUSE'].nunique(), df['ACCCAUSE_LVL1'].nunique(), df['ACCCAUSE_LVL2'].nunique()))

Unique Unfallursachen: 382 & (neue) Unique Unfallursachencluster (Ursachen Level 1): 31 & (neue) Unique Unfallursachencluster (Ursachen Level 2): 7


#### Bestimmung der Anzahl der involvierten und entgleisten Lokomotiven <a class="anchor" id="Lok"></a>
Berechnung der Anzahl der involvierten und entgleisten Lokomotiven mit den 5 jeweiligen Variablen mit der Lokomotivenanzahl

In [18]:
df['LOCOMOTIVES1']=df.apply(lambda row : row['HEADEND1']+row['MIDMAN1']+row['MIDREM1']+row['RMAN1']+row['RREM1'],axis=1)

In [19]:
df['LOCOMOTIVES2']=df.apply(lambda row : row['HEADEND2']+row['MIDMAN2']+row['MIDREM2']+row['RMAN2']+row['RREM2'],axis=1)

In [20]:
# Entfernung der Missing Values für die neue Variable Anzahl der Lokomotiven 
df=df.dropna(subset=['LOCOMOTIVES1'])
df['LOCOMOTIVES1'] = df['LOCOMOTIVES1'].astype(int)

In [21]:
df.shape[0]

96483

#### Berechnung der Geschwindigkeit <a class="anchor" id="spd"></a>
Sollte keine Geschwindigkeit in der Variable trnspd hinterlegt sein, wird die Variable highspd verwendet 

In [22]:
# Erstellen und Ausführen des IterativeImputer Modells zwischen den Variablen TRNSPD und HIGHSPD. Die Ergebnisse werden im DataFrame data_fit gespeichert
df.loc[(df['HIGHSPD']!=0)&(df['TRNSPD']==0), 'TRNSPD'] = np.NaN
data_fit = pd.DataFrame(MICE().fit_transform(df[['TRNSPD', 'HIGHSPD']]))
data_fit.columns = ['TRNSPD', 'HIGHSPD']
data_fit

,TRNSPD,HIGHSPD
0,5.000000,5.0
1,6.000000,6.0
2,5.000000,5.0
3,7.000000,7.0
4,4.000000,4.0
...,...,...
96478,10.000000,10.0
96479,8.000000,8.0
96480,5.000000,8.0
96481,9.852813,10.0


In [23]:
# Abfrage nach der Anzahl der Missing Values 
#(Wert = True)
df['TRNSPD'].isnull().value_counts()

False    84508
True     11975
Name: TRNSPD, dtype: int64

In [24]:
# Einfügen der vorhergesagten Werte
df['SPEED'] = data_fit["TRNSPD"].values

In [25]:
# Anzeigen der Merkmale TRNSPD und HIGHSPD
df[['TRNSPD', 'HIGHSPD']]

,TRNSPD,HIGHSPD
0,5.0,5
1,6.0,6
2,5.0,5
3,7.0,7
4,4.0,4
...,...,...
116203,10.0,10
116204,8.0,8
116205,5.0,8
116208,NaN,10


In [26]:
# Überprüfung der neu berechneten Werte für das Merkmal SPEED
df[df['TRNSPD'].isnull()][['TRNSPD', 'HIGHSPD', 'SPEED']]

,TRNSPD,HIGHSPD,SPEED
5,NaN,4,3.908545
9,NaN,4,3.908545
21,NaN,12,11.834236
38,NaN,2,1.927122
43,NaN,35,34.620598
...,...,...,...
116184,NaN,4,3.908545
116187,NaN,10,9.852813
116199,NaN,5,4.899256
116201,NaN,5,4.899256


#### Berechnung des Infrastrukturschadens anhand der Material- und Streckenschadens <a class="anchor" id="dmg"></a>

In [27]:
# Funktion für die Berechnung des Infrastrukturschadens 
def getInfrastrukturschaden(accdmg, eqpdmg, trkdmg):
    if (accdmg > 0):
        return accdmg
    else:
        return eqpdmg + trkdmg

In [28]:
# Erstellung des Merkmals Inftrastrukturschaden mit den Merkmalen ACCDMG, EQPDMG und TRKDMG
df['INFRASTRUCTURE_DMG']=df.apply(lambda row : getInfrastrukturschaden(row['ACCDMG'],row['EQPDMG'],row['TRKDMG']),axis=1)

In [29]:
df.shape[0]

96483

#### Wetter <a class="anchor" id="weather"></a>

In [30]:
def getWetter(wetter):
    switcher= {
        "1":'Klar',
        "2":'Wolkig',
        "3":'Regen',
        "4":'Nebel',
        "5":'Eisregen',
        "6":'Schnee'  
    }
    return switcher.get(wetter,"NaN")

In [31]:
df['WEATHER'] = df.apply(lambda row : getWetter(row['WEATHER']), axis=1)

In [32]:
df['WEATHER'].isnull().value_counts()

False    96483
Name: WEATHER, dtype: int64

#### Tageszeit <a class="anchor" id="vis"></a>
Ersetzen des Zifferncodes mit den besser lesbaren Namen der Tageszeit

In [33]:
def getVis(vis):
    switcher= {
        "1":'Morgendämmerung',
        "2":'Tag',
        "3":'Abenddämmerung',
        "4":'Abend'
    }
    return switcher.get(vis,"NaN")

In [34]:
df['VISIBLTY'] = df.apply(lambda row : getVis(row['VISIBLTY']), axis=1)

#### Streckenart <a class="anchor" id="Streckenart"></a>
Ersetzen des Zifferncodes mit den besser lesbaren Namen der Streckenart

In [35]:
def getTyptrk(typtrk):
    switcher= {
        1.0 :'Hauptstrecke',
        1 :'Hauptstrecke',
        2.0 :'Depot',
        2 :'Depot',
        3.0 :'Nebengstrecke',
        3 :'Nebenstrecke',
        4.0 :'Industriestrecke',
        4 :'Industriestrecke'
    }
    return switcher.get(typtrk,"NaN")

In [36]:
df.loc[(df['TYPTRK']=='X'), 'TYPTRK'] = np.NaN
df=df.dropna(subset=['TYPTRK'])
df['TYPTRK'] = df['TYPTRK'].astype(int)
df['TYPTRK'] = df.apply(lambda row : getTyptrk(row['TYPTRK']), axis=1)

In [37]:
df.shape[0]

95235

#### Staat <a class="anchor" id="state"></a>
Ersetzen des FIPS Code mit den besser lesbaren Namen der Staaten

In [38]:
def getState(state):
    switcher= {
        1 :'Alabama',
        2 :'Alaska',
        3 :'Amerikanisch Samoa',
        4 :'Arizona',
        5 :'Arkansas',
        6 :'Kalifornien',
        7 :'Kanal',
        8 :'Colorado',
        9 :'Connecticut',
        10 :'Delaware',
        11 :'District of Columbia',
        12 :'Florida',
        13 :'Georgia',
        14 :'Guam',
        15 :'Hawaii',
        16 :'Idaho',
        17 :'Illinois',
        18 :'Indiana',
        19 :'Iowa',
        20 :'Kansas',
        21 :'Kentucky',
        22 :'Louisiana',
        23 :'Maine',
        24 :'Maryland',
        25 :'Massachusetts',
        26 :'Michigan',
        27 :'Minnesota',
        28 :'Mississippi',
        29 :'Missouri',
        30 :'Montana',
        31 :'Nebraska',
        32 :'Nevada',
        33 :'New Hampshire',
        34 :'New Jersey',
        35 :'New Mexico',
        36 :'New York',
        37 :'North Carolina',
        38 :'North Dakota',
        39 :'Ohio',
        40 :'Oklahoma',
        41 :'Oregon',
        42 :'Pennsylvania',
        43 :'Puerto Rico',
        44 :'Rhode Island',
        45 :'South Carolina',
        46 :'South Dakota',
        47 :'Tennessee',
        48 :'Texas',
        49 :'Utah',
        50 :'Vermont',
        51 :'Virginia',
        52 :'Virgin Islands',
        53 :'Washington',
        54 :'West Virginia',
        55 :'Wisconsion',
        56 :'Wyoming'
    }
    return switcher.get(state,"NaN")

In [39]:
df['STATE'] = df['STATE'].astype(int)
df['STATE'] = df.apply(lambda row : getState(row['STATE']), axis=1)

#### Umrechnung des Schadens in Euro <a class="anchor" id="euro"></a>
Erhebliche Laufzeiteinsparungen durch das Erstellen einer eigenen kurz CSV Datei (Mittelwert der nächstliegenden vorhandenen Werte bei fehlenden Kursen)

In [40]:
# Funktion zur Umrechnung der Schäden in Euro
def getEuro(c, damage, year, month, day):
    # print('Schaden: %s & Datum: %s - %s - %s' % (damage, year, month, day))
    damage=c.convert(damage, 'USD', 'EUR', date=date(year, month, day))
    return damage

In [41]:
# Initialisierung des Currency Converters
# Fehlende Wechselkurse werden über eine lineare Hochrechnung ermittel
# Fallback für Daten außerhalb des Zeitraumes
c = CurrencyConverter(fallback_on_missing_rate=True, fallback_on_wrong_date=True)

In [42]:
df=df.dropna(subset=['YEAR4'])
df['YEAR4'] = df['YEAR4'].astype(int)

In [43]:
# Berechnung des Infrastrukturschadens in Euro
df['ACCDMG_EUR'] = df.apply(lambda row : getEuro(c, row['INFRASTRUCTURE_DMG'], row['YEAR4'], row['MONTH'], row['DAY']), axis=1)

#### Bestimmung der Schwere des Unfalls (Nicht Signifikant, Signifikant und Schwer) <a class="anchor" id="sig"></a>

In [44]:
# Funktion für die Bestimmung der Signifikanz des Unfalls
def isSerious(accdmg, killed, injured):
    type="Nicht Signifikant"
    if accdmg>=150000 or injured > 0 or killed > 0:
        type="Signifikant"
        if accdmg>=2000000 or injured >= 5 or killed >0:
            type="Schwer"
    return type

In [45]:
# Ermittlung der Signifikanz des Unfalls mit den Merkmalen Infrastrukturschaden in EUR, Anzahl Tote und Anzahl Verletzte
df["ACCTYPE"] = df.apply(lambda row : isSerious(row['ACCDMG_EUR'],row['RREMPKLD'] + row['PASSKLD'] + row['OTHERKLD'],row['RREMPINJ'] + row['PASSINJ'] + row['OTHERINJ']), axis=1)

In [46]:
df.shape[0]

95235

In [47]:
df['ACCTYPE'].value_counts()

Nicht Signifikant    79487
Signifikant          13722
Schwer                2026
Name: ACCTYPE, dtype: int64

#### Clusterung der Temperaturen <a class="anchor" id="temp"></a>

In [48]:
# Funktion zur Aufteilung der Temperaturen in 5 Gruppen
def clusterTemp(temp):
    cluster = np.array([[32, "Heiß (>= 32° C)"],[20, "Warm (20° bis 31,99° C)"],[0, "Moderat (0° bis 19,99° C)"], [-30, "Kalt (-30° - 0,01° C)"], [-45, "Sehr Kalt (-45° - 30,01° C)"]])
    for x in cluster:
        if ((temp-32)*5/9) >= int(x[0]):
            return x[1]

In [49]:
# Erstellung des neuen Merkmals TEMP_CLUSTER
df['TEMP_CLUSTER'] = df.apply(lambda row : clusterTemp(row["TEMP"]), axis=1)

#### Reduzierung der NaN Values bei Streckenklasse <a class="anchor" id="trkcls"></a>

In [50]:
spd_limit = pd.read_csv('resources/TRACK_SPEED_LIMIT.csv',sep=';',encoding='latin1',dtype = {"TRK_CLASS" : "str"})

In [51]:
def traintype(typeq):
    switcher= {
        "1":'Freight',
        "2":'Passenger',
        "3":'Passenger',
        "4":'Freight',
        "5":'Freight',
        "6":'Freight',
        "7":'Freight',
        "8":'Freight',
        "9":'Freight',
        "A":'Freight',
        "B":'Passenger',
        "C":'Passenger',
        "D":'Passenger',
        "E":'Passenger'  
    }
    return switcher.get(typeq,"NaN")

In [52]:
def getTrkClass(trkclas, trnspd, highspd, typeq):
    if pd.notna(trkclas) and trkclas != "":
        return trkclas
    else :
        if pd.notna(typeq):
            trkclass = "1";
            if highspd>0:
                speed = highspd
            else:
                speed = trnspd
            trntype = traintype (typeq)
            for index, row in spd_limit.iterrows():
                if speed >= row[trntype]:
                    trkclass = row["TRK_CLASS"]
            return trkclass

In [53]:
df['TRKCLAS'].isnull().value_counts()

False    93481
True      1754
Name: TRKCLAS, dtype: int64

In [54]:
df['TRKCLAS'] = df.apply(lambda row : getTrkClass(row['TRKCLAS'], row["TRNSPD"], row["HIGHSPD"], row["TYPEQ"]), axis=1)

In [55]:
df['TRKCLAS'].isnull().value_counts()

False    95126
True       109
Name: TRKCLAS, dtype: int64

#### Überprüfung, ob es zu einer Geschwindigkeitsüberschreitung kam <a class="anchor" id="spd_too_high"></a>

In [56]:
def speedTooHigh(speed,trkclas,typeq):
    if pd.notna(trkclas) and pd.notna(typeq) and pd.notna(speed) and trkclas != '':
        trntype = traintype (typeq)
        limit = spd_limit.loc[spd_limit['TRK_CLASS'] == trkclas, trntype]
        if speed <= limit.values[0]:
            return False
        else:
            return True

In [57]:
df["SPD_TOO_HIGH"] = df.apply(lambda row : speedTooHigh(row["SPEED"], row["TRKCLAS"], row["TYPEQ"]), axis=1)

In [58]:
df['SPD_TOO_HIGH'].isnull().value_counts()

False    88326
True      6909
Name: SPD_TOO_HIGH, dtype: int64

#### Bestimmung der Jahreszeit <a class="anchor" id="jahreszeit"></a>

In [59]:
Y = 2000 # dummy leap year to allow input X-02-29 (leap day)
seasons = [('Winter', (date(Y,  1,  1),  date(Y,  3, 20))),
           ('Frühling', (date(Y,  3, 21),  date(Y,  6, 20))),
           ('Sommer', (date(Y,  6, 21),  date(Y,  9, 22))),
           ('Herbst', (date(Y,  9, 23),  date(Y, 12, 20))),
           ('Winter', (date(Y, 12, 21),  date(Y, 12, 31)))]

In [60]:
def getJahreszeiten(now):
    if isinstance(now, datetime):
        now = now.date()
    now = now.replace(year=Y)
    return next(season for season, (start, end) in seasons
                if start <= now <= end)

In [61]:
df['Jahreszeit'] = df.apply(lambda row : getJahreszeiten(datetime.strptime(str(row['DAY']) + "/" + str(row['MONTH']) + "/" + str(row['YEAR4']), '%d/%m/%Y')), axis=1)

#### Bestimmung der Zugart  <a class="anchor" id="EQUIPMENT_TYPE"></a>

In [62]:
def equipmenttype(typeq): 
    switcher= {
        "1":'Güterzug',
        "2":'Personenzug',
        "3":'Nahverkehrszug',
        "4":'Arbeitszug',
        "5":'Einzelne Wagen',
        "6":'Einzelne Wagen',
        "7":'Depot/Rangieren',
        "8":'Lokomotive',
        "9":'Wartungszug',
        "A":'Wartungszug',
        "B":'Personenzug',
        "C":'Nahverkehrszug',
        "D":'Personenzug',
        "E":'Personenzug'  
    }
    return switcher.get(typeq,"NaN")

In [63]:
df.loc[(df['TYPEQ'].notnull()),'EQUIPMENT_TYPE'] = df.apply(lambda row : equipmenttype(row["TYPEQ"]), axis=1)

In [64]:
df['EQUIPMENT_TYPE'].value_counts()

Güterzug           48501
Depot/Rangieren    24802
Einzelne Wagen      6443
Personenzug         3556
Lokomotive          3428
Nahverkehrszug       611
Arbeitszug           546
Wartungszug          439
Name: EQUIPMENT_TYPE, dtype: int64

#### Ersetzen der MissingValues in der Spalte Zuggewicht <a class="anchor" id="TONS"></a>
Die MV Werte in der Spalte TONS werden anhand einer Regression mit der Anzahl der leeren und beladenen Wagons ermittelt

In [65]:
df.loc[(df['EQUIPMENT_TYPE']!='Locomotive')&(df['TONS']==0), 'TONS'] = np.NaN

**Korrelationsanalyse der Wagons gegenüber des Zuggewichtes**

In [66]:
df[df['EQUIPMENT_TYPE']!='Locomotive'][['INCDTNO','TONS','LOADF1', 'LOADP1', 'EMPTYF1', 'EMPTYP1', 'LOADF2', 'EMPTYF2', 'LOADP2', 'EMPTYP2']].corr()

,TONS,LOADF1,LOADP1,EMPTYF1,EMPTYP1,LOADF2,EMPTYF2,LOADP2,EMPTYP2
TONS,1.000000,0.809320,-0.016189,-0.051267,-0.011209,0.326347,-0.065324,-0.015614,-0.008901
LOADF1,0.809320,1.000000,-0.126643,-0.048710,-0.086957,0.478319,-0.051576,-0.045432,-0.057734
LOADP1,-0.016189,-0.126643,1.000000,-0.088790,-0.002417,-0.076259,-0.053538,0.350640,-0.000074
EMPTYF1,-0.051267,-0.048710,-0.088790,1.000000,-0.060414,-0.101451,0.439558,-0.031659,-0.039239
EMPTYP1,-0.011209,-0.086957,-0.002417,-0.060414,1.000000,-0.054401,-0.032860,0.000847,0.469875
LOADF2,0.326347,0.478319,-0.076259,-0.101451,-0.054401,1.000000,-0.010344,-0.025885,-0.034764
EMPTYF2,-0.065324,-0.051576,-0.053538,0.439558,-0.032860,-0.010344,1.000000,-0.019676,-0.022186
LOADP2,-0.015614,-0.045432,0.350640,-0.031659,0.000847,-0.025885,-0.019676,1.000000,0.008370
EMPTYP2,-0.008901,-0.057734,-0.000074,-0.039239,0.469875,-0.034764,-0.022186,0.008370,1.000000


In [67]:
df[df['EQUIPMENT_TYPE']!='Locomotive'][['INCDTNO','TONS','LOADF1', 'LOADP1', 'EMPTYF1', 'EMPTYP1', 'LOADF2', 'EMPTYF2', 'LOADP2', 'EMPTYP2']]

,INCDTNO,TONS,LOADF1,LOADP1,EMPTYF1,EMPTYP1,LOADF2,EMPTYF2,LOADP2,EMPTYP2
0,LA1904,7100.0,56,0,56,0,0,3,0,0
1,RE902,NaN,37,0,1,0,3,0,0,0
2,YKR0490,250.0,2,0,0,0,2,0,0,0
3,G1693,8744.0,75,0,12,0,5,0,0,0
4,G1694,NaN,2,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
116203,240189101,2500.0,38,0,0,0,0,0,0,0
116204,12889,NaN,1,0,0,0,1,0,0,0
116205,12889,NaN,20,0,50,0,0,0,0,0
116208,012589A,NaN,0,0,0,0,0,0,0,0


In [68]:
data_fit = pd.DataFrame(MICE().fit_transform(df[['TONS', 'LOADF1', 'LOADP1', 'EMPTYF1', 'EMPTYP1', 'LOADF2', 'EMPTYF2', 'LOADP2', 'EMPTYP2']]))
data_fit.columns = ['TONS', 'LOADF1', 'LOADP1', 'EMPTYF1', 'EMPTYP1', 'LOADF2', 'EMPTYF2', 'LOADP2', 'EMPTYP2']

In [69]:
def negValue(tons):
    if tons < 0: 
        return 0
    else :
        return tons

In [70]:
data_fit['TONS'] = data_fit.apply(lambda row : negValue(row["TONS"]), axis=1)

In [71]:
df['TONS'] = data_fit["TONS"].values

In [72]:
df[['INCDTNO','TONS','LOADF1', 'LOADP1', 'EMPTYF1', 'EMPTYP1', 'LOADF2', 'EMPTYF2']]

,INCDTNO,TONS,LOADF1,LOADP1,EMPTYF1,EMPTYP1,LOADF2,EMPTYF2
0,LA1904,7100.000000,56,0,56,0,0,3
1,RE902,4333.188214,37,0,1,0,3,0
2,YKR0490,250.000000,2,0,0,0,2,0
3,G1693,8744.000000,75,0,12,0,5,0
4,G1694,262.404823,2,0,0,0,0,0
...,...,...,...,...,...,...,...,...
116203,240189101,2500.000000,38,0,0,0,0,0
116204,12889,168.256439,1,0,0,0,1,0
116205,12889,3832.358975,20,0,50,0,0,0
116208,012589A,34.889974,0,0,0,0,0,0


In [73]:
df["EQPDMG"].isna().value_counts()

False    95235
Name: EQPDMG, dtype: int64

In [74]:
#df.loc[((df['ACCDMG']==0)|(df['ACCDMG'].isna()))&((df['TRKDMG']==0)|(df['TRKDMG'].isna()))&((df['EQPDMG']==0)|(df['EQPDMG'].isna())), 'TRKDMG'] = np.NaN
#df.loc[((df['ACCDMG']==0)|(df['ACCDMG'].isna()))&((df['TRKDMG']==0)|(df['TRKDMG'].isna()))&((df['EQPDMG']==0)|(df['EQPDMG'].isna())), 'EQPDMG'] = np.NaN
#df.loc[((df['ACCDMG']==0)|(df['ACCDMG'].isna()))&((df['TRKDMG']==0)|(df['TRKDMG'].isna()))&((df['EQPDMG']==0)|(df['EQPDMG'].isna())), 'INFRASTRUCTURE_DMG'] = np.NaN

In [75]:
df["EQPDMG"].isna().value_counts()

False    95235
Name: EQPDMG, dtype: int64

In [76]:
df["INFRASTRUCTURE_DMG"].isna().value_counts()

False    95235
Name: INFRASTRUCTURE_DMG, dtype: int64

#### Entfernung aller Unfälle mit mehr als einen beteiligten Zug <a class="anchor" id="entfernen_mit_mehr_zügen"></a>

In [77]:
df.shape[0]

95235

In [78]:
multi_train_df = df[df.duplicated(subset=["INCDTNO","YEAR4","MONTH","DAY"], keep=False)].copy()

In [79]:
multi_train_df['INCDTNO'].shape[0]

17618

In [80]:
single_train_df=df.drop_duplicates(subset=["INCDTNO","YEAR4","MONTH","DAY"], keep=False).copy()

In [81]:
single_train_df['INCDTNO'].shape[0]

77617

In [82]:
True if (df.shape[0]==(multi_train_df['INCDTNO'].shape[0]+single_train_df['INCDTNO'].shape[0])) else False

True

#### Entfernung aller NaN Werte <a class="anchor" id="nan"></a>

In [83]:
single_train_df.shape[0]

77617

In [84]:
single_train_df=single_train_df.dropna(subset=['TRKCLAS', 'EQUIPMENT_TYPE', 'TYPTRK', 'ACCIDENT_TYPE', 'STATE', 'WEATHER', 'VISIBLTY', 'Jahreszeit', 'TONS', 'ACCCAUSE_LVL1', 'SPEED', 'ACCCAUSE_LVL2', 'LOCOMOTIVES1', 'INFRASTRUCTURE_DMG'])
single_train_df.shape[0]

70759

In [85]:
#single_train_df.to_csv (r'resources/accident_data.csv', index = False, header=True, sep=';')



#### Mehrere beteiligte Züge <a class="anchor" id="multi"></a>


In [86]:
train_count = multi_train_df.groupby(["INCDTNO","YEAR4","MONTH","DAY"])["INCDTNO"].count()

In [87]:
def getTrainCount(incdtno, year, month, day):
    return train_count[incdtno, year, month, day]

In [88]:
multi_train_df['NUMBER_OF_TRAINS'] = multi_train_df.apply(lambda row : getTrainCount(row['INCDTNO'], row['YEAR4'], row['MONTH'], row['DAY']),axis=1)

In [89]:
multi_train_df = multi_train_df.sort_values(by='EQUIPMENT_TYPE', ascending=True)
multi_train_group = multi_train_df.groupby(["INCDTNO","YEAR4","MONTH","DAY"], as_index=False)
multi_train_list = multi_train_group.agg({'ACCDMG' : 'mean','INFRASTRUCTURE_DMG' : 'mean', 'TRKDMG' : 'sum', 'EQPDMG' : 'sum', 
                 'LOCOMOTIVES1': 'sum','LOCOMOTIVES2': 'sum', 'RREMPKLD': 'sum', 'RREMPINJ': 'sum', 
                 'PASSKLD': 'sum', 'PASSINJ': 'sum', 'OTHERKLD': 'sum', 'OTHERINJ': 'sum', 'SPEED' : 'mean', 
                 'LOADF1': 'sum', 'LOADF1': 'sum', 'LOADF2': 'sum', 'LOADF2': 'sum', 'EMPTYF1': 'sum', 
                 'EMPTYP1' : 'sum', 'LOADP1' : 'sum',
                 'EMPTYF1': 'sum', 'EMPTYF2': 'sum', 'EMPTYF2': 'sum', 'TONS': 'sum', 'TYPE' : 'first', 
                 'TYPEQ' : 'first', 'TRKCLAS' : 'first', 'TYPTRK' : 'first', 'ACCAUSE' : 'first', 
                 'TEMP' : 'first', 'TEMP_CLUSTER' : 'first', 'VISIBLTY' : 'first', 'WEATHER' : 'first', 'STATE' : 'first', 
                 'Jahreszeit' : 'first', 'ACCIDENT_TYPE' : 'first', 'ACCCAUSE_LVL1' : 'first', 'ACCCAUSE_LVL2' : 'first', 
                                          'EQUIPMENT_TYPE' : lambda text: ''.join(text.to_string(index=False)).replace(' ', '').replace('\n', ', ').replace('Train', ' Train').replace('CutofCars', 'Cut of Cars').replace('SingleCar', 'Single Car')})
multi_train_list = multi_train_list.reset_index()
multi_train_list['INFRASTRUCTURE_DMG']=multi_train_list.apply(lambda row : getInfrastrukturschaden(row['ACCDMG'],row['EQPDMG'],row['TRKDMG']),axis=1)
multi_train_list['ACCDMG_EUR'] = multi_train_list.apply(lambda row : getEuro(c, row['INFRASTRUCTURE_DMG'], row['YEAR4'], row['MONTH'], row['DAY']), axis=1)
multi_train_list["ACCTYPE"] = multi_train_list.apply(lambda row : isSerious(row['ACCDMG_EUR'],row['RREMPKLD'] + row['PASSKLD'] + row['OTHERKLD'],row['RREMPINJ'] + row['PASSINJ'] + row['OTHERINJ']), axis=1)
multi_train_list["SPD_TOO_HIGH"] = multi_train_list.apply(lambda row : speedTooHigh(row["SPEED"], row["TRKCLAS"], row["TYPEQ"]), axis=1)

In [90]:
multi_train_list['EQUIPMENT_TYPE'].isnull().value_counts()

False    8612
Name: EQUIPMENT_TYPE, dtype: int64

In [91]:
multi_train_list=multi_train_list.dropna(subset=['TRKCLAS', 'EQUIPMENT_TYPE', 'TYPTRK', 'ACCIDENT_TYPE', 'STATE', 'WEATHER', 'VISIBLTY', 'Jahreszeit', 'TONS', 'ACCCAUSE_LVL1', 'SPEED', 'ACCCAUSE_LVL2', 'LOCOMOTIVES1', 'INFRASTRUCTURE_DMG'])
multi_train_list.shape[0]
multi_train_list.loc[((multi_train_list['ACCDMG']>0)|(multi_train_list['ACCDMG'].isna()))&((multi_train_list['TRKDMG']==0)|(multi_train_list['TRKDMG'].isna()))&((multi_train_list['EQPDMG']==0)|(multi_train_list['EQPDMG'].isna())), 'TRKDMG'] = np.NaN
multi_train_list.loc[((multi_train_list['ACCDMG']>0)|(multi_train_list['ACCDMG'].isna()))&((multi_train_list['TRKDMG']==0)|(multi_train_list['TRKDMG'].isna()))&((multi_train_list['EQPDMG']==0)|(multi_train_list['EQPDMG'].isna())), 'EQPDMG'] = np.NaN
multi_train_list.loc[((multi_train_list['ACCDMG']>0)|(multi_train_list['ACCDMG'].isna()))&((multi_train_list['TRKDMG']==0)|(multi_train_list['TRKDMG'].isna()))&((multi_train_list['EQPDMG']==0)|(multi_train_list['EQPDMG'].isna())), 'INFRASTRUCTURE_DMG'] = np.NaN

In [92]:
multi_train_list.to_csv (r'resources/multi_train_accident_data.csv', index = False, header=True, sep=';')